In [1]:

import os
# !pip install numpy==1.23.5
# !pip install 'monai[all]'
import shutil
import tempfile
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from monai.utils import first, set_determinism
from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    LoadImaged,
    ScaleIntensityRanged,

)
from monai.visualize import blend_images, matshow3d, plot_2d_or_3d_image
from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import UNETR

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    Dataset,
)

from google.colab import drive
import os
import json
import random
import torch

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.6.0+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /usr/local/lib/python3.11/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.3
Nibabel version: 5.3.2
scikit-image version: 0.25.2
scipy version: 1.14.1
Pillow version: 11.1.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.21.0+cu124
tqdm version: 4.67.1
lmdb version: 1.6.2
psutil version: 5.9.5
pandas version: 2.2.2
einops version: 0.8.1
transformers version: 4.51.3
mlflow version: 2.21.3
pynrrd version: 1.1.3
clearml version: 2.0.0rc0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [ ]:

import os
import json
from sklearn.model_selection import train_test_split

image_dir = "/content/drive/MyDrive/input_spine/images"
label_dir = "/content/drive/MyDrive/input_spine/masks"
output_json = "/content/drive/MyDrive/input_spine/spider_dataset.json"

# Get base filenames (without extension) that exist in both dirs
image_files = set(os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith(".mha"))
label_files = set(os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith(".mha"))

common_files = sorted(list(image_files & label_files))  # valid pairs

# Build data dicts
all_data = [
    {
        "image": f"input_spine/images/{name}.mha",
        "label": f"input_spine/masks/{name}.mha"
    }
    for name in common_files
]

# Split into train/val/test (80/10/10)
train_data, test_val = train_test_split(all_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_val, test_size=0.5, random_state=42)

# Build final dataset dict
dataset_dict = {
    "name": "SpineDataset",
    "description": "Spine MRI t1 and t2 segmentation",
    "labels": {
        "0": "background",
        "1": "1",
        "2": "2",
        "3": "3",
        "4": "4",
        "5": "5",
        "6": "6",
        "7": "7",
        "8": "8",
        "9": "9",
        "10": "10",
        "11": "11",
        "12": "12",
        "13": "13",
        "14": "14",
        "15": "15",
        "16": "16",
        "17": "17",
        "18": "18",
        "19": "19"
    },
    "training": train_data,
    "validation": val_data,
    "test": test_data
}

# Save
with open(output_json, "w") as f:
    json.dump(dataset_dict, f, indent=4)

print(f"JSON saved with {len(train_data)} train, {len(val_data)} val, {len(test_data)} test samples.")



JSON saved with 357 train, 45 val, 45 test samples.


In [5]:
drive.mount('/content/drive')

IMAGE_DIR = f"/input_spine/images"
LABEL_DIR = f"/input_spine/masks"
OUTPUT_PATH = f"/content/dataset.json"


Mounted at /content/drive


In [6]:

from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    ResizeWithPadOrCropd, ToTensord
)

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    ResizeWithPadOrCropd(
        keys=["image", "label"],
        spatial_size=(32, 128, 128)
    ),
    ScaleIntensityd(keys=["image"]),  # Normalization for image only
    ToTensord(keys=["image", "label"])
])


In [ ]:
# from monai.data import CacheDataset, DataLoader, load_decathlon_datalist, pad_list_data_collate
from monai.transforms import Resized

import torch

split_json = "/content/drive/MyDrive/input_spine/spider_dataset (2).json"
data_dir = "/content/drive/MyDrive/" 
datalist_file=data_dir+split_json
# Load splits using updated data_dir
train_files = load_decathlon_datalist(split_json, True, data_list_key="training", base_dir=data_dir)
val_files = load_decathlon_datalist(split_json, True, data_list_key="validation", base_dir=data_dir)

# Define your transforms
train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    Resized(keys=["image", "label"], spatial_size=(32, 128, 128), mode=["trilinear", "nearest"]),
    ScaleIntensityd(keys=["image"]),
    ToTensord(keys=["image", "label"])
])
# Create cache dataset
train_ds = CacheDataset(
    data=train_files,
    transform=train_transforms,
    cache_num=24,
    cache_rate=1.0,
    num_workers=2,
)

train_loader = DataLoader(
    train_ds, batch_size=4, shuffle=True, num_workers=2,
    pin_memory=True, collate_fn=pad_list_data_collate
)

val_ds = CacheDataset(
    data=val_files,
    transform=train_transforms,
    cache_num=2,
    cache_rate=1.0,
    num_workers=4,
)

val_loader = DataLoader(
    val_ds, batch_size=2, shuffle=False, num_workers=2,
    pin_memory=True, collate_fn=pad_list_data_collate
)

# Check sample label stats
for batch in DataLoader(val_ds, batch_size=1):
    label = batch["label"]
    max_label = label.max()
    unique_labels = torch.unique(label)
    print(f"Max label: {max_label}, Unique: {unique_labels}")
    break


Loading dataset: 100%|██████████| 2/2 [00:04<00:00,  2.26s/it]

Max label: 205.0, Unique: tensor([  0.,   1.,   2.,   3.,   4.,   5., 100., 201., 202., 203., 204., 205.])


In [10]:
#Check the shape of input data after preprocessing and blending image with mask
check_ds = Dataset(data=train_files, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)

ret = blend_images(image=check_data["image"][0], label=check_data["label"][0], alpha=0.5, cmap="jet", rescale_arrays=True)
print(ret.shape)

torch.Size([3, 32, 128, 128])


In [ ]:
#setting the device to use GPU and if not available use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')

#Define UNETR Model
model = UNETR(
    in_channels=1,                      
    out_channels=20,                     
    img_size=(32, 128, 128),            
    feature_size=16,                   
    hidden_size=768,                    
    mlp_dim=3072,                      
    num_heads=12,                       
    #pos_embed="perceptron",
    norm_name="instance",               
    res_block=True,               
).to(device)

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
torch.backends.cudnn.benchmark = True

#Using AdamW optimizer which includes weight decay for regularization
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
from monai.transforms import EnsureType, EnsureTyped  # Add this line

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# For predictions
post_pred = Compose([
    AsDiscrete(argmax=True, to_onehot=20),
    EnsureType(data_type="tensor")
])

# For labels
post_label = Compose([
    AsDiscrete(to_onehot=20),
    EnsureType(data_type="tensor")
])

dice_metric = DiceMetric(include_background=False, reduction="mean")

def validation(epoch_iterator_val):
    """
    Evaluates the model on the validation dataset and computes the mean Dice score.

    Args:
        epoch_iterator_val (iterable): Iterator for the validation dataset.

    Returns:
        float: The mean Dice score for the validation dataset.
    """
    model.eval()
    with torch.no_grad():
        for batch in epoch_iterator_val:
            val_inputs, val_labels = (batch["image"].to(device), batch["label"].to(device))

            val_labels = remap_labels(val_labels)  # <-- Critical fix

            val_outputs = sliding_window_inference(val_inputs, (32, 128, 128), 4, model)

            val_labels_list = decollate_batch(val_labels)
            val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]

            val_outputs_list = decollate_batch(val_outputs)
            val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]

            dice_metric(y_pred=val_output_convert, y=val_labels_convert)

            # Update progress bar
            epoch_iterator_val.set_description(f"Validate ({global_step} / {10.0} Steps)")  # noqa: B038

        # Get the mean Dice score for the validation set
        mean_dice_val = dice_metric.aggregate().item()
        dice_metric.reset() 

    return mean_dice_val
label_map = {
    0:0,   1:1,   2:2,   3:3,   4:4,   5:5,   6:6,   7:7,   8:8,   9:9, 100:10., 201:11,
        202:12, 203:13, 204:14, 205:15, 206:16, 207:17, 208:18, 209:19
}

# Vectorized remap
def remap_labels(label_tensor):
    label_tensor = label_tensor.clone()
    for original, mapped in label_map.items():
        label_tensor[label_tensor == original] = mapped
    return label_tensor


# Define training function
def train(global_step, train_loader, dice_val_best, global_step_best):
    """
    Trains the model for one epoch and evaluates it periodically.

    Args:
        global_step (int): The current global step count.
        train_loader (DataLoader): DataLoader for the training dataset.
        dice_val_best (float): The best Dice validation score achieved so far.
        global_step_best (int): The global step at which the best Dice score was achieved.

    Returns:
        tuple: Updated global step, best Dice validation score, and global step at which the best score was achieved.
    """
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True)

    for step, batch in enumerate(epoch_iterator):
        step += 1
        x = batch["image"].to(device)
        y = batch["label"].to(device)
        y = remap_labels(y)
        # print(f"Input shape: {x.shape}, Label shape: {y.shape}")

        if not torch.all((y >= 0) & (y <= 19)):
            print(f"Invalid labels found. Min: {y.min()}, Max: {y.max()}")
            raise ValueError("Found label values outside expected range [0, 12].")

        # Forward pass
        logit_map = model(x)
        loss = loss_function(logit_map, y)

        # Backprop
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()

        epoch_iterator.set_description(  # noqa: B038
            "Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss)
        )

        # Periodic validation
        if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations:
            epoch_iterator_val = tqdm(val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True)

            try:
                # Just to peek at what's happening
                sample_batch = next(iter(val_loader))
                sample_img, sample_lbl = sample_batch["image"].to(device), sample_batch["label"].to(device)
                sample_lbl = remap_labels(sample_lbl)
                sample_output = model(sample_img)

                print("logits shape:", sample_output.shape)
                print("label shape:", sample_lbl.shape)
                print("logits min/max:", sample_output.min().item(), sample_output.max().item())
                print("label min/max:", sample_lbl.min().item(), sample_lbl.max().item())
            except Exception as e:
                print(f"Debug print failed: {e}")

            dice_val = validation(epoch_iterator_val)
            epoch_loss /= step
            epoch_loss_values.append(epoch_loss)
            metric_values.append(dice_val)

            # Save model if best
            if dice_val > dice_val_best:
                print("Model Was Saved! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val))
                dice_val_best = dice_val
                global_step_best = global_step
                torch.save(model.state_dict(), os.path.join(root_dir, "best_metric_model.pth"))
            else:
                print("Model Not Saved! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val))

        global_step += 1

    return global_step, dice_val_best, global_step_best

max_iterations = 21000                            
eval_num = 1000                                    
post_label = AsDiscrete(to_onehot=20)              
post_pred = AsDiscrete(argmax=True, to_onehot=20)   
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False) 
global_step = 0                                     
dice_val_best = 0.0                                
global_step_best = 0                                
epoch_loss_values = []                            
metric_values = []                                 


In [ ]:
root_dir='/content/'
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(global_step, train_loader, dice_val_best, global_step_best)
model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))

Validate (X / X Steps) (dice=X.X):   0%|          | 0/23 [00:00<?, ?it/s]

logits shape: torch.Size([2, 13, 32, 128, 128])
label shape: torch.Size([2, 1, 32, 128, 128])
logits min/max: -2.5234076976776123 3.5678296089172363
label min/max: 0.0 11.0



Validate (990 / 10.0 Steps): 100%|██████████| 23/23 [00:51<00:00,  2.26s/it]


Model Was Saved! Current Best Avg. Dice: 0.0 Current Avg. Dice: 0.16584041714668274


Validate (X / X Steps) (dice=X.X):   0%|          | 0/23 [00:00<?, ?it/s]

...

Validate (X / X Steps) (dice=X.X):   0%|          | 0/23 [00:00<?, ?it/s]

logits shape: torch.Size([2, 13, 32, 128, 128])
label shape: torch.Size([2, 1, 32, 128, 128])
logits min/max: -23.270769119262695 14.185359001159668
label min/max: 0.0 11.0



Validate (10980 / 10.0 Steps): 100%|██████████| 23/23 [00:13<00:00,  1.74it/s]


Model Was Saved! Current Best Avg. Dice: 0.6602839231491089 Current Avg. Dice: 0.6816810965538025


Training (11012 / 21000 Steps) (loss=0.53990):  37%|███▋      | 33/90 [01:07<01:19,  1.39s/it]

In [ ]:
print(os.path.exists('/content/drive/MyDrive/input_spine/input_spine/images/'))

False
